In [40]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import random
from fake_useragent import UserAgent

In [41]:
ua = UserAgent()
headers = {'User-Agent': ua.random}

base_url = "https://uk.trustpilot.com/review/www.vodafone.co.uk?page="

reviews = []
seen_reviews = set() 


In [42]:
for page in range(1, 3):  # Adjust number of pages as needed
    url = base_url + str(page)
    print(f"Scraping page {page}...")
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    for i, card in enumerate(soup.select('article')):
        if i < 4:
            continue  # 🚫 skip sticky duplicates

        try:
            rating = card.select_one('[data-service-review-rating]').get('data-service-review-rating')
        except:
            rating = None
        
        try:
            review = card.select_one('p.typography_body-l__v5JLj').text.strip()
        except:
            review = None

        try:
             # Try to get full name first
            name_tag = card.select_one('span[data-consumer-name-typography]')
            if name_tag:
                name = name_tag.text.strip()
            else:
                # Fallback to initials if full name missing
                name = card.select_one('span.typography_heading-xs__osRhC').text.strip()
        except:
            name = None

        try:
            location = card.select_one('span.typography_body-m__k2UI7').text.strip()
        except:
            location = None

        try:
            date = card.select_one('time')['datetime']
        except:
            date = None

        reviews.append({
            'Rating': rating,
            'Review': review,
            'Name': name,
            'Location': location,
            'Date': date
        })

    time.sleep(1)

df = pd.DataFrame(reviews)

Scraping page 1...
Scraping page 2...


In [43]:
df

,Rating,Review,Name,Location,Date
0,1,"Trying to buy broadband through Uswitch, then ...",Alfie Calas,GB,2025-06-05T22:03:24.000Z
1,5,Sona did a great job either my trade in and co...,Julliette,GB,2025-06-05T21:51:17.000Z
2,1,"One of the worst , if not the worst when it co...",Vlad Ureche,GB,2025-06-05T21:28:36.000Z
3,1,How those people get 4.6 rate that’s a joke my...,Adam Farbotko,GB,2025-06-05T21:14:08.000Z
4,1,Held to ransom by a ‘reputable’ company. Purch...,Chloe,GB,2025-06-05T20:24:59.000Z
5,5,Excellent customer service. Vinnie was very he...,Farida Ariori,GB,2025-06-05T20:05:30.000Z
6,5,Excellent service from Rushabh! He explained e...,kawater alismaeel,GB,2025-06-05T19:52:59.000Z
7,5,Steven listened then provided a number of opti...,Dylan Owen,GB,2025-06-05T19:43:30.000Z
8,5,Great experience at Vodafone today sorting my ...,Maia,GB,2025-06-05T19:10:06.000Z
9,5,Shaun explained everything and was very helpfu...,James Hogan,GB,2025-06-05T18:56:03.000Z


In [44]:
df.Location.value_counts()

Location
GB    39
US     1
Name: count, dtype: int64

In [45]:
df.Review.head(1)

0    Trying to buy broadband through Uswitch, then ...
Name: Review, dtype: object